In [ ]:
# !pip install import-ipynb

In [ ]:
import pandas as pd
import import_ipynb
import preprocessamentofuncoes as pp

In [ ]:
covidData = pd.read_csv("../../database/opendatasus/sp-px-opendata-2022.csv", on_bad_lines='skip')

In [ ]:
# Visualizando Dados
# covidData.head()
covidData.shape

In [ ]:
# 'dataPrimeiraDose', 'dataSegundaDose'
colunasImportantes = ['source_id', 'sexo', 'racaCor', 'idade', 'sintomas', 'condicoes', 
                      'classificacaoFinal', 'evolucaoCaso']
covidData = covidData[colunasImportantes]

### Limpeza de Dados - Classificação Final


In [ ]:
# Visualizando o formato das condições
# covidData['classificacaoFinal'].value_counts()

In [ ]:
covidData = pp.classFinPreprocessamento(covidData, 'classificacaoFinal')

In [ ]:
# covidData['classificacaoFinal'].value_counts()

In [ ]:
dados_faltosos = pp.visualizarQntFaltosos(covidData)
dados_faltosos

### Limpeza de Dados - Atibuto Evolução

In [ ]:
# covidData[alvo].unique()
evolucao = 'evolucaoCaso'
covidData[evolucao].value_counts()

In [ ]:
covidData = pp.evolucaoPreprocessamento(covidData, evolucao)
covidData[evolucao].value_counts()

In [ ]:
# dados_faltosos = pp.visualizarQntFaltosos(covidData)
# dados_faltosos

In [ ]:
# covidData[evolucao].value_counts()
# covidData.shape

### Limpeza de Dados - Sintomas

In [ ]:
# covidData.head()
# covidData.shape

In [ ]:
# covidData[alvo].unique()
sintomas = 'sintomas'
covidData = pp.sintomasPreprocessamento(covidData, sintomas)
covidData.shape

In [ ]:
# covidData.head()
# 'Febre' in covidData['sintomas'] 

### Limpeza de Dados - Condições

In [ ]:
# covidData.shape

In [ ]:
condicoes = 'condicoes'
covidData = pp.condicoesPreprocessamento(covidData, condicoes)
covidData.shape

In [ ]:
# pp.condicoesQntValores(covidData)
# covidData.head()

### Limpeza de Dados - Idade

In [ ]:
covidData.shape

In [ ]:
idade = 'idade'
covidData = pp.idadePreprocessamento(covidData, idade, 'faixaetaria')

In [ ]:
covidData['faixaetaria'].value_counts()

### Limpeza de Dados - Raça

In [ ]:
raca = 'racaCor'
covidData = pp.racaPreprocessamento(covidData, raca)
covidData[raca].value_counts()

### Limpeza de Dados - Sexo

In [ ]:
sexo = 'sexo'
covidData[sexo].value_counts()

In [ ]:
covidData = pp.sexoPreprocessamento(covidData, sexo)

In [ ]:
covidData[sexo].value_counts()

### Limpeza dos Dados - Removendo Colunas

In [ ]:
# source_id, idade, sintomas, classificacaoFinal
colunasRemovidas = ['source_id', 'idade', 'sintomas', 'classificacaoFinal']
covidData.shape

In [ ]:
covidData = pp.removerColunas(covidData, colunasRemovidas)
covidData.shape

### Limpeza dos Dados - Formatando Colunas

In [ ]:
# Colunas Formatadas: sexo e racaCor
print(covidData['sexo'].value_counts(), '\n\n',covidData['racaCor'].value_counts())

In [ ]:
covidData = formatarColunas(covidData)
print(covidData['sexo'].value_counts(), '\n\n',covidData['racaCor'].value_counts())